### 📌 검색기(Retriever) 개요
- Retriever: **쿼리 → 관련 문서 반환** 역할
- VectorStore 위에 얹어서 사용
- 다양한 변형:
  - Contextual Compression
  - Ensemble
  - Parent/Child 문서 검색
  - MultiQuery, MultiVector
  - TimeWeighted 등
- 👉 RAG 파이프라인에서 **정확한 맥락 추출의 핵심 모듈**

In [1]:
# # 01. 검색기(Retriever) 개요
# from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import FAISS

# texts = ["강아지는 충성스럽다", "고양이는 귀엽다", "호랑이는 멋지다", "사자는 숲의 왕이다"]
# embeddings = OpenAIEmbeddings()

# # 벡터스토어 생성
# vectorstore = FAISS.from_texts(texts, embeddings)

# # 기본 검색기
# retriever = vectorstore.as_retriever()
# results = retriever.invoke("집에서 많이 키우는 동물")

# for r in results:
#     print(r.page_content)

### 📌 벡터스토어 기반 검색기
- 가장 기본적인 Retriever
- 내부적으로 **벡터 유사도 검색** 수행
- 옵션:
  - `k`: 검색 결과 개수
  - `search_type`: similarity, mmr 등

In [3]:
# # 02. 벡터스토어 기반 검색기 (VectorStore-backed Retriever)
# retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
# results = retriever.invoke("멋진 동물")

# for r in results:
#     print(r.page_content)

### 📌 ContextualCompressionRetriever
- 기본 검색 결과에서 불필요한 문맥 제거
- **LLM 압축기**를 이용해 핵심 내용만 반환
- 긴 문서 요약 후 전달 시 유용

In [5]:
# # 03. 문서 압축기 (ContextualCompressionRetriever)
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import LLMChainExtractor
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo")
# compressor = LLMChainExtractor.from_llm(llm)

# compression_retriever = ContextualCompressionRetriever(
#     base_retriever=retriever, 
#     base_compressor=compressor
# )

# results = compression_retriever.invoke("동물 특징 요약")
# for r in results:
#     print(r.page_content)

### 📌 EnsembleRetriever
- 여러 검색기 결합
  - 예: VectorStore + BM25(키워드)
- 서로 다른 검색 전략을 **가중치 기반 앙상블**
- 정확도 ↑, 다양한 문맥 포착


In [6]:
# # 04. 앙상블 검색기 (EnsembleRetriever)
# from langchain.retrievers import EnsembleRetriever
# from langchain.retrievers import BM25Retriever

# # 키워드 기반 검색기
# bm25_retriever = BM25Retriever.from_texts(texts)

# # 앙상블 (벡터 + 키워드)
# ensemble_retriever = EnsembleRetriever(
#     retrievers=[retriever, bm25_retriever],
#     weights=[0.5, 0.5]
# )

# results = ensemble_retriever.invoke("집에서 기르는 동물")
# for r in results:
#     print(r.page_content)

### 📌 LongContextReorder
- 긴 문서를 **순서 최적화**하여 모델 입력에 더 잘 맞도록 조정
- 맥락 유지를 위한 재정렬 (예: 앞뒤 순서 바꿔치기)

In [7]:
# # 05. 긴 문서 재정렬 (LongContextReorder)
# from langchain.retrievers import LongContextReorder

# reordering = LongContextReorder()
# docs = retriever.invoke("동물")  # 벡터 검색 결과
# reordered_docs = reordering.transform_documents(docs)

# for r in reordered_docs:
#     print(r.page_content)

### 📌 ParentDocumentRetriever
- **작은 청크로 검색** 후 → **원래 부모 문서 단위**로 반환
- 장점: 검색은 세밀하게, 출력은 문맥 넓게

In [8]:
# # 06. 부모 문서 검색기 (ParentDocumentRetriever)
# from langchain.storage import InMemoryStore
# from langchain.retrievers import ParentDocumentRetriever
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# # 작은 chunk로 분할한 후 parent_id를 연결
# child_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)
# parent_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)

# docstore = InMemoryStore()
# retriever = ParentDocumentRetriever(
#     vectorstore=FAISS.from_texts(texts, embeddings),
#     docstore=docstore,
#     child_splitter=child_splitter,
#     parent_splitter=parent_splitter,
# )

# retriever.add_texts(texts)

# results = retriever.invoke("귀여운 동물")
# for r in results:
#     print(r.page_content)

### 📌 MultiQueryRetriever
- LLM이 원 쿼리를 **다양한 표현으로 변환**
- 예: "반려동물" → "집에서 기르는 동물", "애완동물"
- Recall ↑ (더 많은 후보 검색)

In [9]:
# # 07. 다중 쿼리 생성 검색기 (MultiQueryRetriever)
# from langchain.retrievers.multi_query import MultiQueryRetriever

# multi_query_retriever = MultiQueryRetriever.from_llm(
#     retriever=retriever,
#     llm=llm
# )

# results = multi_query_retriever.invoke("집에서 키우는 동물")
# for r in results:
#     print(r.page_content)

### 📌 MultiVectorRetriever
- 한 문서에 대해 **여러 벡터**를 연결
- 예: 긴 문서를 다양한 방식으로 임베딩
- Recall ↑, 문맥 다양성 반영

In [10]:
# # 08. 다중 벡터 검색기 (MultiVectorRetriever)
# from langchain.retrievers.multi_vector import MultiVectorRetriever

# docstore = InMemoryStore()
# vectorstore = FAISS.from_texts(texts, embeddings)

# multi_vector_retriever = MultiVectorRetriever(
#     vectorstore=vectorstore,
#     docstore=docstore,
# )

# multi_vector_retriever.add_texts(["강아지는 집에서 사람과 함께 산다."], ids=["doc1"])

# results = multi_vector_retriever.invoke("사람과 함께 사는 동물")
# for r in results:
#     print(r.page_content)

### 📌 Self-Query Retriever
- **쿼리에서 메타데이터 필터 자동 생성**
- LLM이 쿼리 해석 → VectorStore 필터링에 반영
- 예: "2021년 이후 논문" → `{year > 2021}`

In [11]:
# # 09. Self-Query Retriever
# from langchain.retrievers.self_query.base import SelfQueryRetriever
# from langchain.chains.query_constructor.schema import AttributeInfo
# from langchain.prompts import PromptTemplate

# metadata_field_info = [
#     AttributeInfo(name="category", description="문서의 주제", type="string"),
# ]

# retriever = SelfQueryRetriever.from_llm(
#     llm, retriever=vectorstore.as_retriever(), metadata_field_info=metadata_field_info, document_content_description="동물"
# )

# results = retriever.invoke("카테고리가 동물인 문서 보여줘")
# for r in results:
#     print(r.page_content)

### 📌 TimeWeightedVectorStoreRetriever
- 벡터 유사도 + **시간 가중치** 함께 반영
- 최근 정보일수록 더 높은 점수 부여
- 파라미터:
  - `decay_rate`: 시간에 따라 중요도 감소 속도
  - `k`: 검색 결과 개수
- 👉 **시간 민감한 대화/뉴스/로그 검색**에 최적

In [12]:
# # 10. TimeWeightedVectorStoreRetriever
# from datetime import datetime, timedelta
# from langchain.retrievers import TimeWeightedVectorStoreRetriever

# # 벡터스토어 + 시간가중치 기반
# vectorstore = FAISS.from_texts(["오늘 점심은 피자", "지난주에 파스타"], embeddings)

# retriever = TimeWeightedVectorStoreRetriever(
#     vectorstore=vectorstore,
#     embeddings=embeddings,
#     decay_rate=0.01,   # 시간 경과 시 가중치 감소
#     k=2
# )

# # 문서 저장 (메타데이터에 timestamp 추가)
# retriever.add_documents([
#     {"page_content": "오늘 아침은 빵", "metadata": {"last_accessed_at": datetime.now()}},
#     {"page_content": "어제 저녁은 치킨", "metadata": {"last_accessed_at": datetime.now() - timedelta(days=1)}}
# ])

# results = retriever.invoke("식사")
# for r in results:
#     print(r.page_content, r.metadata)

# ✅ 최종 정리
- **VectorStore Retriever**: 기본 벡터 검색
- **ContextualCompressionRetriever**: 긴 문서 압축
- **EnsembleRetriever**: 다중 검색기 결합
- **LongContextReorder**: 긴 문서 순서 최적화
- **ParentDocumentRetriever**: 검색은 작은 단위, 반환은 부모 단위
- **MultiQueryRetriever**: 쿼리 다양화
- **MultiVectorRetriever**: 문서 → 다중 벡터 매핑
- **Self-QueryRetriever**: LLM 기반 메타데이터 필터링
- **TimeWeightedRetriever**: 시간 가중치 기반 검색

👉 상황에 맞는 Retriever 선택이 RAG 성능을 크게 좌우